In [72]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from keras import models
from keras import layers
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score


load data

In [2]:
df=pd.read_csv('editing.csv')

In [3]:
df.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,...,market_segment_type_0,market_segment_type_1,market_segment_type_2,market_segment_type_3,market_segment_type_4,has_children,co1,co2,co3,co4
0,2,0,1,4,0,118,2017,12,28,0,...,0,1,0,0,0,0,0,0,0,1
1,2,1,0,2,0,17,2018,4,14,0,...,0,1,0,0,0,1,0,1,0,0
2,1,0,1,5,0,349,2018,10,4,0,...,1,0,0,0,0,0,0,0,0,1
3,1,0,2,4,0,69,2018,6,12,0,...,1,0,0,0,0,0,0,1,0,0
4,2,0,0,4,0,11,2018,1,20,0,...,0,1,0,0,0,0,1,0,0,0


split data


In [4]:
X=df.drop(columns=['booking_status'])
y=df['booking_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
y_train.shape ,y_test.shape

((10519,), (3507,))

In [5]:
NOcanceled =y_train.sum()
total=y_train.shape[0]
print('Percentage of cancel in train set',NOcanceled/total)
NOcanceled =y_test.sum()
total=y_test.shape[0]
print('Percentage of cancel in test set',NOcanceled/total)

Percentage of cancel in train set 0.2905219127293469
Percentage of cancel in test set 0.2968349016253208


we have outlliers in our data set but we think it is acceptable because there some event the price of hotel ancreased and You must book very early before you arrive Although we well tray modeling in both data with outlier and without it

In [6]:
X_trainOut=X_train.copy()
y_trainOut=y_train.copy()

remove outliers

In [7]:
def remove_outliers(x,y,column):
    Q1 = x[column].quantile(0.25)
    Q3 = x[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5*IQR
    y=y[(x[column] >= lower_bound) & (x[column] <= upper_bound)]
    x=x[(x[column] >= lower_bound) & (x[column] <= upper_bound)]
    return x,y

for col in ['lead_time','avg_price_per_room']:
    
    X_trainOut ,y_trainOut= remove_outliers(X_trainOut,y_trainOut,col)
    
X_train.shape, X_trainOut.shape,y_train.shape, y_trainOut.shape

((10519, 35), (9770, 35), (10519,), (9770,))

In [8]:
NOcanceled =y_trainOut.sum()
total=y_trainOut.shape[0]
print('Percentage of cancel in train set without outlier',NOcanceled/total)

Percentage of cancel in train set without outlier 0.2804503582395087


we can see hear the Percentage of cancel after removing the outliers decreased Which is expected to affect the model negatively

data scaling

In [9]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
scaler = MinMaxScaler()
X_trainOut = scaler.fit_transform(X_trainOut)
X_testOut = scaler.transform(X_test)

c:\Users\OMARS\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


now let's start building different models for both data with outliers and without

Logistic Regression

In [11]:
log=LogisticRegression()
log.fit(X_train,y_train)
yd=log.predict(X_train)
print('train accuracy:',accuracy_score(yd,y_train))
yed=log.predict(X_test)
print('test accuracy:',accuracy_score(yed,y_test))

train accuracy: 0.8069208099629243
test accuracy: 0.8006843455945253


c:\Users\OMARS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
log=LogisticRegression()
log.fit(X_trainOut,y_trainOut)
yd=log.predict(X_trainOut)
print('train accuracy:',accuracy_score(yd,y_trainOut))
yed=log.predict(X_testOut)
print('test accuracy:',accuracy_score(yed,y_test))

train accuracy: 0.8056294779938588
test accuracy: 0.7031650983746792


c:\Users\OMARS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Random Forest Classifier

In [13]:
rfc = RandomForestClassifier(n_estimators=100)
param_grid = {'max_depth': range(1, 31)}

grid_search = GridSearchCV(rfc, param_grid=param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Accuracy on testing data: ", grid_search.score(X_test, y_test))

Best parameters:  {'max_depth': 15}
Accuracy on testing data:  0.8511548331907614


In [14]:
rfc = RandomForestClassifier(n_estimators=300,max_depth=15)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_train)
print( 'train accuracy:' ,accuracy_score(y_train, y_pred))
y_pred = rfc.predict(X_test)
print('test accuracy:' ,accuracy_score(y_test, y_pred))

train accuracy: 0.9044585987261147
test accuracy: 0.8534359851725121


In [15]:
rfc = RandomForestClassifier(n_estimators=100)
param_grid = {'max_depth': range(1, 31)}

grid_search = GridSearchCV(rfc, param_grid=param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_trainOut, y_trainOut)

print("Best parameters: ", grid_search.best_params_)
print("Accuracy on testing data: ", grid_search.score(X_test, y_test))

Best parameters:  {'max_depth': 15}
Accuracy on testing data:  0.7627601938979185


In [16]:
rfc = RandomForestClassifier(n_estimators=300,max_depth=15)
rfc.fit(X_trainOut, y_trainOut)
y_pred = rfc.predict(X_trainOut)
print( 'train accuracy:' ,accuracy_score(y_trainOut, y_pred))
y_pred = rfc.predict(X_testOut)
print('test accuracy:' ,accuracy_score(y_test, y_pred))

train accuracy: 0.9192425793244626
test accuracy: 0.7031650983746792


KNeighborsClassifier

In [17]:
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': range(1, 11)}
grid_search = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best value of n_neighbors:", grid_search.best_params_)

accuracy = grid_search.score(X_test, y_test)
print("Accuracy of KNN classifier with best n_neighbors:", accuracy)


Best value of n_neighbors: {'n_neighbors': 9}
Accuracy of KNN classifier with best n_neighbors: 0.799258625605931


In [18]:
knn = KNeighborsClassifier(n_neighbors=9)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_train)
print('train accuracy:' ,accuracy_score(y_train, y_pred))
y_pred = knn.predict(X_test)
print('test accuracy:' ,accuracy_score(y_test, y_pred))

train accuracy: 0.8342998383876794
test accuracy: 0.799258625605931


In [41]:
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': range(1, 11)}
grid_search = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_trainOut, y_trainOut)

print("Best value of n_neighbors:", grid_search.best_params_)

accuracy = grid_search.score(X_testOut, y_test)
print("Accuracy of KNN classifier with best n_neighbors:", accuracy)

Best value of n_neighbors: {'n_neighbors': 9}
Accuracy of KNN classifier with best n_neighbors: 0.6227544910179641


In [ ]:
knn = KNeighborsClassifier(n_neighbors=9)

knn.fit(X_trainOut, y_trainOut)

y_pred = knn.predict(X_trainOut)
print('train accuracy:' ,accuracy_score(y_trainOut, y_pred))
y_pred = knn.predict(X_testOut)
print('test accuracy:' ,accuracy_score(y_test, y_pred))

train accuracy: 0.8425793244626407
test accuracy: 0.6227544910179641


support vector machine (poly)

In [42]:
for i in range(2,11):
    svmc = SVC(kernel='poly',degree=i)
    svmc.fit(X_train, y_train)

    y_pred = svmc.predict(X_train)
    print('degree = ',i)
    accuracy = accuracy_score(y_train, y_pred)
    print('train accuracy:' ,accuracy)

    y_pred = svmc.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print('test accuracy:' ,accuracy)

degree =  2
train accuracy: 0.815381690274741
test accuracy: 0.8058169375534645
degree =  3
train accuracy: 0.8179484741895617
test accuracy: 0.8092386655260907
degree =  4
train accuracy: 0.8240327027283962
test accuracy: 0.8118049615055604
degree =  5
train accuracy: 0.8319231866146972
test accuracy: 0.8166524094667807
degree =  6
train accuracy: 0.8393383401464017
test accuracy: 0.8160821214713431
degree =  7
train accuracy: 0.8460880311816713
test accuracy: 0.8163672654690619
degree =  8
train accuracy: 0.8505561365148778
test accuracy: 0.8177929854576561
degree =  9
train accuracy: 0.8548341097062458
test accuracy: 0.8166524094667807
degree =  10
train accuracy: 0.8607282061032417
test accuracy: 0.8157969774736242


In [43]:
for i in range(2,10):
    svmc = SVC(kernel='poly',degree=i)
    svmc.fit(X_trainOut, y_trainOut)

    y_pred = svmc.predict(X_trainOut)
    print('degree = ',i)
    accuracy = accuracy_score(y_trainOut, y_pred)
    print('train accuracy:' ,accuracy)

    y_pred = svmc.predict(X_testOut)

    accuracy = accuracy_score(y_test, y_pred)
    print('test accuracy:' ,accuracy)

degree =  2
train accuracy: 0.8165813715455476
test accuracy: 0.7031650983746792
degree =  3
train accuracy: 0.8204708290685773
test accuracy: 0.2968349016253208
degree =  4
train accuracy: 0.8295803480040942
test accuracy: 0.7031650983746792
degree =  5
train accuracy: 0.8363357215967246
test accuracy: 0.2968349016253208
degree =  6
train accuracy: 0.8441146366427841
test accuracy: 0.7031650983746792
degree =  7
train accuracy: 0.8506653019447288
test accuracy: 0.2968349016253208
degree =  8
train accuracy: 0.8569089048106449
test accuracy: 0.7031650983746792
degree =  9
train accuracy: 0.8623336745138178
test accuracy: 0.2968349016253208


In [ ]:
gb_clf = GradientBoostingClassifier(n_estimators=100,random_state=40)

param_grid = {'learning_rate': [0.1],'max_depth':range(2, 10), 'max_features':range(4, 12)}
grid_search = GridSearchCV(gb_clf, param_grid, cv=5, scoring='accuracy',verbose=0)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)

accuracy = grid_search.score(X_test, y_test)
print("Accuracy of Gradient Boosting Classifier with best parameters:", accuracy) 

Best parameters: {'learning_rate': 0.1, 'max_depth': 6, 'max_features': 9}
Accuracy of Gradient Boosting Classifier with best parameters: 0.8545765611633875


In [ ]:
gb_clf = GradientBoostingClassifier(n_estimators=300, learning_rate=0.1, max_features=9, max_depth=6, random_state=40)

gb_clf.fit(X_train, y_train)

y_pred = gb_clf.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print('train accuracy:' ,accuracy) 

y_pred = gb_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('test accuracy:' ,accuracy) 

train accuracy: 0.9360205342713186
test accuracy: 0.8545765611633875


In [ ]:
gb_clf = GradientBoostingClassifier(n_estimators=100,random_state=40)

param_grid = {'learning_rate': [0.1],'max_depth':range(2, 10), 'max_features':range(4, 12)}
grid_search = GridSearchCV(gb_clf, param_grid, cv=5, scoring='accuracy',verbose=0)
grid_search.fit(X_trainOut, y_trainOut)

print("Best parameters:", grid_search.best_params_)

accuracy = grid_search.score(X_testOut, y_test)
print("Accuracy of Gradient Boosting Classifier with best parameters:", accuracy) 

Best parameters: {'learning_rate': 0.1, 'max_depth': 7, 'max_features': 11}
Accuracy of Gradient Boosting Classifier with best parameters: 0.7031650983746792


In [ ]:
gb_clf = GradientBoostingClassifier(n_estimators=300, learning_rate=0.1, max_features=11, max_depth=6, random_state=40)

gb_clf.fit(X_trainOut, y_trainOut)

y_pred = gb_clf.predict(X_trainOut)
accuracy = accuracy_score(y_trainOut, y_pred)
print('train accuracy:' ,accuracy) 

y_pred = gb_clf.predict(X_testOut)
accuracy = accuracy_score(y_test, y_pred)
print('test accuracy:' ,accuracy) 

train accuracy: 0.8831115660184238
test accuracy: 0.7031650983746792


neural network

In [44]:
partial_X_train , X_val , partial_y_train , y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [45]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu',input_dim=partial_X_train.shape[1]))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', patience=30, mode='max')

In [46]:
history = model.fit(partial_X_train,partial_y_train, epochs=1000, batch_size=512, callbacks=[es] , validation_data=(X_val, y_val))


Epoch 1/1000
17/17 [==============================] - 1s 12ms/step - loss: 0.6123 - accuracy: 0.7031 - val_loss: 0.5803 - val_accuracy: 0.7106
Epoch 2/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.5658 - accuracy: 0.7092 - val_loss: 0.5459 - val_accuracy: 0.7106
Epoch 3/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.5364 - accuracy: 0.7133 - val_loss: 0.5160 - val_accuracy: 0.7191
Epoch 4/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.5110 - accuracy: 0.7409 - val_loss: 0.4895 - val_accuracy: 0.7581
Epoch 5/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.4877 - accuracy: 0.7611 - val_loss: 0.4672 - val_accuracy: 0.7856
Epoch 6/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.4691 - accuracy: 0.7754 - val_loss: 0.4519 - val_accuracy: 0.8009
Epoch 7/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.4548 - accuracy: 0.7813 - val_loss: 0.4361 - val_accuracy: 0.7942
Epoch

In [71]:
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate the accuracy on the test data
accuracy = (y_pred_binary[:,0]== y_test).mean()
print("Test Accuracy:", accuracy)

110/110 [==============================] - 0s 1ms/step
[1 0 1 ... 1 1 0]
(3507,)
(3507,)
Test Accuracy: 0.8072426575420587


In [ ]:
#cross vlatidion 

#insamble 

# Feature Engering

the best two models is GradientBoostingClassifier and RandomForestClassifier

Voting Classifier

In [74]:
clf1 = GradientBoostingClassifier(n_estimators=300, learning_rate=0.1, max_features=9, max_depth=6, random_state=40)
clf2 = RandomForestClassifier(n_estimators=300,max_depth=14)

voting_clf = VotingClassifier(estimators=[('gb', clf1), ('rf', clf2)], voting='soft')

cv_scores = cross_val_score(voting_clf, X_train, y_train, cv=5)
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())

voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_train)

accuracy = accuracy_score(y_train, y_pred)
print("train accuracy:", accuracy)

y_pred = voting_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)

Cross-validation scores: [0.86121673 0.8493346  0.85361217 0.85551331 0.8554446 ]
Mean cross-validation score: 0.8550242818063639
train accuracy: 0.9245175396900847
Test accuracy: 0.8565725691474194
